In [1]:
import os
import re
from tqdm.auto import tqdm
import time
import gc
import geopandas as gpd
import rasterio
from rasterio.crs import CRS
from imageprocessing import convert_SHPtoPNG

/Users/banoczymartin/OE Projects/Mapping Upper Canopy Level/venvs_can/canopysegmentation/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folder = '../data/2024-03-16_dataset/standard/original'
out_folder = '../data/2024-03-16_dataset/standard/formatted'

In [13]:
def dropna_SHPs(folder, pattern = r'^tile_shp.*\.shp$'):
    '''
    returns: 
        -   runtime
        -   number of removed image-mask pairs
    '''
    
    ad_extensions = ['.cpg','.dbf','.prj','.shx']
    
    start = time.time()
    
    bin_list = []
    
    filenames = os.listdir(folder)
    total_length = len(filenames)
    with tqdm(total=total_length) as pbar:
        for filename in filenames:
            if re.match(pattern, filename):
                shapefile = gpd.read_file(os.path.join(folder,filename))
                if len(shapefile.geometry)==0:
                    splitted = filename.split('.')[0].split('_')
                    bin_list.append((splitted[2],splitted[3]))
                    #bin_list.append((filename.split('.')[0].split('_')[2],filename.split('.')[0].split('_')[3]))
                    os.remove(os.path.join(folder,filename))
                    os.remove(os.path.join(folder,filename.replace('shp','tif')))
                    for ext in ad_extensions:
                        if os.path.exists(os.path.join(folder,filename[:-4]+ext)):
                            os.remove(os.path.join(folder,filename[:-4]+ext))
                            #print()
            pbar.update(1)
            
    end = time.time()
    runtime = end-start
    gc.collect()
    return runtime, bin_list

runtime, bin_list = dropna_SHPs(folder)
print(runtime)
print(bin_list)

100%|██████████| 5466/5466 [00:09<00:00, 558.68it/s]

9.837299823760986
[]


In [175]:
print(len(bin_list))

839


In [14]:
bin_list.__contains__(('12','10'))

False

In [7]:

#with rasterio.open('../data/big_wetransfer_2023_02_23-bakonyszucs-ecw_2024-03-10_1733/2023-02-23_Bakonyszucs.tif') as src:
with rasterio.open('../data/actual_data_l/2023-02-23_Bakonyszucs_actual.tif','r+') as src:
    print(src.crs)
    if src.crs == None:
        src.crs = CRS.from_epsg(23700)

EPSG:23700


In [17]:
def filter_invalid_CRS(folder,pattern = r'^tile_tif.*\.tif$'):
    out_list = []
    filenames = os.listdir(folder)
    total_length = len(filenames)
    with tqdm(total=total_length) as pbar:
        for filename in filenames:
            if re.match(pattern, filename):
                with rasterio.open(os.path.join(folder,filename)) as src:
                    if src.crs == None:
                        print(filename)    
            pbar.update(1)            
filter_invalid_CRS(folder)

100%|██████████| 5466/5466 [00:02<00:00, 1966.99it/s]


In [15]:
gpd.read_file('../data/2024-03-16_dataset/standard/original/tile_tif_12_10.tif').crs

DriverError: '../data/2024-03-16_dataset/standard/original/tile_tif_12_10.tif' not recognized as a supported file format.

In [3]:
files = os.listdir(folder)
for file in files:
    if file.endswith('.shp'):
        tif_path = os.path.join(folder,str(file).replace('_shp_','_tif_').replace('.shp','.tif'))
        shp_path = os.path.join(folder,file)
        out_path = os.path.join(out_folder, os.path.splitext(file)[0]+'.png')
        convert_SHPtoPNG(tif_path, shp_path, out_path, (500,500), 5)

../data/2024-03-16_dataset/standard/original/tile_shp_12_10.shp
../data/2024-03-16_dataset/standard/original/tile_tif_12_10.tif


AttributeError: 'MultiPoint' object has no attribute 'x'